# Lab 7: Human motion generation

## Advanced deep learning

## Setup

### Setup dataset


Please access this Google Drive folder: [link](https://drive.google.com/drive/folders/1V5yzlwBPSNVPj33SfDHnvMykXISh3CyB?usp=sharing) and create a shortcut in the root of your Google Drive `/content/drive/MyDrive/`.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/humanml3d-data

Mounted at /content/drive
caption_clip  caption_raw  checkpoints	humanml3d_test_split.txt  smplh  smpl_rifke


### Setup environment

Make sure you're running on a T4 GPU Colab instance; if not, activate it.

In [2]:
!nvidia-smi

Tue Mar  4 19:29:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
torch.cuda.is_available()

True

Clone the lab repository.

In [4]:
!git clone https://github.com/robincourant/lab7-CSC52087EP.git
#!git clone https://github.com/robincourant/lab-MotionDiT.git

Cloning into 'lab7-CSC52087EP'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 78 (delta 5), reused 78 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (78/78), 1.64 MiB | 26.63 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [5]:
%cd lab5-CSC52087EP
#%cd lab-MotionDiT
!ln -s /content/drive/MyDrive/humanml3d-data ./ # Plug the dataset in the repo

[Errno 2] No such file or directory: 'lab5-CSC52087EP'
/content


Install required libraries

In [6]:
!pip install hydra-core
!pip install pyrender
!pip install smplx
!pip install torchtyping
!pip install lightning
!pip install ema_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.4 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=42870f753fc9f97cdcd2f7c6885d2c8121b63fe3fb97ce4f91ca30898cb7d8d9
  Stored in directory: /root/.cache/pip/wheels/1a/97/32/461f837398029ad76911109f07047fde1d7b661a147c7c56d1
Successfully built antlr4-python3-runtime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.1/962.1 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.6/708.6 kB 50.7 MB/s eta 0:00:00
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745193 sha256=09bf76b96718d02f098880c41903fa0648dfbd3b15fc422f06079f85d05cd065
  Stored in directory: /root/.cache/pip/wheels/2f/37/f5/f88cd3dddf75bc3ce608e44bf8a79078c408bf1f351a50818e
Successfully built PyOpenGL
  Attempting uninstall: PyOpenGL
    Found existing installation: PyOpenGL 3.1.9
    Uninstalling PyOpenGL-3.1.9:
      Successfully uninstalled PyOpenGL-3.1.9
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Human motion dataset and representation

### HumanML3D dataset

#### Question 1:
​The HumanML3D dataset includes 14,616 movements and 44,970 text descriptions

### SMPL representation


#### Question 2:
Pose Parameters (θ): These define the 3D rotations of the body's joints.

Shape Parameters (β): These capture the variations in body shape, such as height and body proportions.

#### Code 2
*Complete `visualize_smpl.py`*

In [7]:
!HYDRA_FULL_ERROR=1 PYTHONPATH=$(pwd) python src/visualize_smpl.py

import moviepy.editor
moviepy.editor.ipython_display("./smpl.mp4")

[2025-03-04 19:37:43,490][numexpr.utils][INFO] - NumExpr defaulting to 2 threads.
/content/src/datasets/modalities/human_dataset.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental fe

  if event.key is 'enter':



## Model architectures

### Config A: Incontext

#### Code 3
*Complete `src/models/modules/incontext.py`*

In [8]:
!PYTHONPATH=$(pwd) python src/models/modules/incontext.py

Test passed!


### Config B: AdaLN

#### Question 3:
$$
\text{adaLN}(x, \gamma, \beta) = \gamma \cdot \frac{x - \mu}{\sigma + \epsilon} + \beta
$$

#### Code 4
*Complete `src/models/modules/adaln.py`*

In [9]:
!PYTHONPATH=$(pwd) python src/models/modules/adaln.py

Test passed!


### Config C: Cross attention

#### Question 4:
$$
\text{CA}(\mathbf{x}, \mathbf{c}) = \text{softmax} \left( \frac{\mathbf{Q} \mathbf{K}^T}{\sqrt{d}} \right) \mathbf{V}
$$

#### Code 5
*Complete `src/models/modules/cross attention.py`*

In [37]:
!PYTHONPATH=$(pwd) python src/models/modules/cross_attention.py

Test passed!


#### Question 5:
AdaLN is more computationally efficient and memory-friendly than cross-attention, as it directly modulates features using adaptive scaling and shifting, avoiding costly attention score computations

## Diffusion framework

### DDPM

#### Code 6
*Complete `src/training/losses/ddpm.py`*

In [38]:
!PYTHONPATH=$(pwd) python src/training/losses/ddpm.py

Test passed!


#### Code 7
*Complete `src/training/sampler/ddpm.py`*

In [40]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/sampler@diffuser.test_sampler=ddpm seed=2

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_incontext.mp4")

Seed set to 2
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have 

### DDIM

#### Question 6:
DDIM makes image generation faster and more deterministic than DDPM because it removes random noise at each step.

#### Code 8
*Complete `src/training/sampler/ddim.py`*

In [45]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/sampler@diffuser.test_sampler=ddim seed=2

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddim_incontext.mp4")

Seed set to 2
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have 

#### Question 7:
*Answer here...*

## Result analysis

### Qualitative analysis

#### Code 9

In [ ]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=incontext \
checkpoint_path=./humanml3d-data/checkpoints/incontext.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_incontext.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have

In [ ]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=adaln \
checkpoint_path=./humanml3d-data/checkpoints/adaln.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_adaln.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have

In [ ]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=cross_attention \
checkpoint_path=./humanml3d-data/checkpoints/cross_attention.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_cross_attention.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have

#### Question 8
*Answer here...*

### Quantitative analysis

#### Question 9:
The main assumption when computing the Fréchet Distance is that both the reference and generated feature spaces follow a multivariate Gaussian distribution

#### Code 10
*Complete src/metrics/frechet.py*

#### Code 11
*Complete src/metrics/similarity.py*

#### Code 12

#### Bonus 1:
*Answer here...*

In [ ]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=incontext \
checkpoint_path=./humanml3d-data/checkpoints/incontext.ckpt

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
[2025-03-03 23:47:23,385][OpenGL.acceleratesupport][INFO] - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
/content/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user 

In [ ]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=adaln \
checkpoint_path=./humanml3d-data/checkpoints/adaln.ckpt

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
[2025-03-04 00:05:20,792][OpenGL.acceleratesupport][INFO] - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
/content/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user 

In [ ]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=cross_attention \
checkpoint_path=./humanml3d-data/checkpoints/cross_attention.ckpt

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
[2025-03-04 00:13:31,099][OpenGL.acceleratesupport][INFO] - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
/content/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user 

#### Question 10:
The results show that the model in context showed better metrics in contrast to cross attention and adaIn

#### Bonus 2:
*Answer here...*